<div style="display: flex; align-items: left;">
    <a href="https://sites.google.com/corp/google.com/genai-solutions/home?authuser=0">
        <img src="https://storage.googleapis.com/miscfilespublic/Linkedin%20Banner%20%E2%80%93%202.png" style="margin-right">
    </a>
</div>

In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# **Open Data QnA: Set up Dataset on CloudSQL for PostgreSQL**

---

This notebook shows how to copy a BigQuery public dataset to CloudSQL for PostgreSQL


This is accomplished through the three following steps: 
> i. Set up PostgreSQL instance and databae on Google Cloud SQL

> ii. Copy the BigQuery table to Cloud Storage Bucket

> iii. Create the table in PostgreSQL databae using csv file in Cloud Storage Bucket



## 🔗 **1. Connect Your Google Cloud Project**
Time to connect your Google Cloud Project to this notebook. 

In [1]:
#@markdown Please fill in the value below with your GCP project ID and then run the cell.
PROJECT_ID = 'talk2data-genai-sa'

# Quick input validations.
assert PROJECT_ID, "⚠️ Please provide your Google Cloud Project ID"

# Configure gcloud.
!gcloud config set project {PROJECT_ID}
print(f'Project has been set to {PROJECT_ID}')
# !gcloud auth application-default set-quota-project {PROJECT_ID}



Updated property [core/project].
Project has been set to talk2data-genai-sa

Credentials saved to file: [/home/srilakshmil/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "talk2data-genai-sa" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the project owning the resource.


## 🔐 **2. Authenticate to Google Cloud**
Authenticate to Google Cloud as the IAM user logged into this notebook in order to access your Google Cloud Project.

You can do this within Google Colab or using the Application Default Credentials in the Google Cloud CLI.

In [2]:
"""Colab Auth""" 
# from google.colab import auth
# auth.authenticate_user()


"""Google CLI Auth"""
# !gcloud auth application-default login


"""Jupiter Notebook Auth"""
import google.auth
credentials, project_id = google.auth.default()
# credentials = google.auth.credentials.with_scopes_if_required(credentials)
# authed_http = google.auth.transport.requests.AuthorizedSession(credentials)

import os
os.environ['GOOGLE_CLOUD_QUOTA_PROJECT']=PROJECT_ID
os.environ['GOOGLE_CLOUD_PROJECT']=PROJECT_ID

## ☁️ **3. Set up Cloud SQL PostgreSQL Instance** 
A **Postgres** Cloud SQL instance is required for the following stages of this notebook.

To connect and access our Postgres Cloud SQL database instance(s) we will leverage the [Cloud SQL Python Connector](https://github.com/GoogleCloudPlatform/cloud-sql-python-connector).

The Cloud SQL Python Connector is a library that can be used alongside a database driver to allow users to easily connect to a Cloud SQL database without having to manually allowlist IP or manage SSL certificates. 

💽 **Create a Postgres Instance**

Running the below cell will verify the existence of a Cloud SQL instance or create a new one if one does not exist.

> ⏳ - Creating a Cloud SQL instance may take a few minutes.

In [19]:
#@markdown Please fill in the both the Google Cloud region and name of your Cloud SQL instance. Once filled in, run the cell.

# Below are the recommended defaults; These can be changed to values of your choice
PG_REGION = "us-central1" #@param {type:"string"}
PG_INSTANCE = "pg15-opendataqna-src"
PG_DATABASE = "opendataqna-src-db"
PG_USER = "data_user"
PG_PASSWORD = "data123"

# Quick input validations.
assert PG_REGION, "us-central1"
assert PG_INSTANCE, "pg15-opendataqna-src"

# check if Cloud SQL instance exists in the provided region and create it if it does not exist
database_version = !gcloud sql instances describe {PG_INSTANCE} --format="value(databaseVersion)"
if database_version[0].startswith("POSTGRES"):
  print("Found existing Postgres Cloud SQL Instance!")
else:
  print("Creating new Cloud SQL instance...")
  !gcloud sql instances create {PG_INSTANCE} --database-version=POSTGRES_15 \
    --region={PG_REGION} --cpu=1 --memory=4GB --root-password={PG_PASSWORD} \
    --database-flags=cloudsql.iam_authentication=On

# Create a database on the instance and a user with password
!gcloud sql databases create  {PG_DATABASE} --instance={PG_INSTANCE}
!gcloud sql users create {PG_USER} \
--instance={PG_INSTANCE} \
--password={PG_PASSWORD}

Found existing Postgres Cloud SQL Instance!
Creating Cloud SQL database...done.                                            
Created database [opendataqna-src-db].
instance: pg15-opendataqna-src
name: opendataqna-src-db
project: talk2data-genai-sa
Creating Cloud SQL user...done.                                                
Created user [data_user].


## ➡️ **4. Migrate a public BigQuery database to PostgreSQL instance**
Let's migrate a public BigQuery dataset over to the newly created PostgreSQL instance. 

### A) Set up a Google Cloud Storage Bucket 
This bucket will be used to store the exported BigQuery public dataset.

In [20]:
from google.cloud import storage
from urllib.error import HTTPError

# Choose a name for the bucket; You might have to choose a different name if the name below already exists
BUCKET_NAME = str(PROJECT_ID+'-opendataqna') #@param {type:"string"} 

# Creating a bucket
storage_client = storage.Client(project=PROJECT_ID)

try: 
    bucket = storage_client.bucket(BUCKET_NAME)

    if bucket.exists(): 
        print("This bucket already exists.")

    else:
        bucket = storage_client.create_bucket(BUCKET_NAME)
        print(f"Bucket {bucket.name} created")

except:
        print("⚠️ This bucket already exists in another project. Make sure to give your bucket a unique name.")

This bucket already exists.


### B) Export BigQuery Dataset to the Bucket


In [13]:
#@markdown Please choose a BigQuery Public dataset to export. You can leave the default values. Once filled in, run the cell.

# Below are the recommended defaults; You may choose a different database to export
BQ_PROJECT = "bigquery-public-data"
BQ_DATABASE = "google_dei"
BQ_TABLE = "dar_intersectional_hiring"


# Quick input validations.
assert BQ_PROJECT, "⚠️ Please specify the BigQuery Project"
assert BQ_DATABASE, "⚠️ Please specify the BigQuery Database"
assert BQ_TABLE, "⚠️ Please specify the BigQuery Table"


# Export the bigquery data to Google Bucket
BUCKET_FILENAME = f"{BQ_TABLE}.csv" #@param {type:"string"}


from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

destination_uri = "gs://{}/{}".format(BUCKET_NAME, BUCKET_FILENAME)
dataset_ref = bigquery.DatasetReference(BQ_PROJECT, BQ_DATABASE)
table_ref = dataset_ref.table(BQ_TABLE)

extract_job = client.extract_table(
    table_ref,
    destination_uri,
    # Location must match that of the source table.
    location="US",
)  # API request
extract_job.result()  # Waits for job to complete.

print(
    f"Exported {BQ_PROJECT}:{BQ_DATABASE}.{BQ_TABLE} to {destination_uri}"
)

Exported bigquery-public-data:google_dei.dar_intersectional_hiring to gs://talk2data-genai-sa-opendataqna/dar_intersectional_hiring.csv


### C) Retrieve Data Types and Formats 
To migrate the exported .csv files to PostgreSQL, we need to fetch the Data Types and Format from the exported csv file.
This needs to be done as we're setting up the PostgreSQL table and columns first (and need to provide the columns in the setup).
We will load the .csv content into the table afterwards. 

In [25]:
import pandas as pd

df = pd.read_csv(destination_uri)
field_names = df.columns
print(f'Column Names: {field_names}\n')
field_types = df.dtypes
print(f'Column Names: {field_types}')


Column Names: Index(['workforce', 'report_year', 'gender_us', 'race_asian', 'race_black',
       'race_hispanic_latinx', 'race_native_american', 'race_white'],
      dtype='object')

Column Names: workforce                object
report_year               int64
gender_us                object
race_asian              float64
race_black              float64
race_hispanic_latinx    float64
race_native_american    float64
race_white              float64
dtype: object


### D) Build the SQL Query for Table Creation 
Every database is different. To acommodate for different table structures depending on which BigQuery dataset is being loaded in, we will build the SQL query for creating the required PostgreSQL table dynamically. 

In [26]:
def get_sql(pg_schema, bq_table, field_names, field_types): 

    cols = "" 

    for i in range(len(field_names)): 
        cols += str(field_names[i]) +" "+ str(field_types[i])
        if i < (len(field_names)-1): 
            cols += ", "


    sql = f"""CREATE TABLE {pg_schema}.{bq_table}({cols})"""

    return sql

#@markdown Please specify the PGSchema or leave it as default (public)
PG_SCHEMA = BQ_DATABASE
sql = get_sql(PG_SCHEMA, BQ_TABLE, field_names, field_types)
print(f'\n\nsql for creating the required PostgreSQL table: \n\n{sql} \n\n')



sql for creating the required PostgreSQL table: 

CREATE TABLE google_dei.dar_intersectional_hiring(workforce object, report_year int64, gender_us object, race_asian float64, race_black float64, race_hispanic_latinx float64, race_native_american float64, race_white float64) 




/tmp/ipykernel_19271/3703849647.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cols += str(field_names[i]) +" "+ str(field_types[i])


### E) Create the PostgreSQL Table

In [27]:
import asyncio 

async def create_pg_table(PROJECT_ID,
                          PG_REGION,
                          PG_INSTANCE,
                          PG_PASSWORD,
                          BQ_TABLE, 
                          PG_DATABASE, 
                          SQL,
                          PG_USER): 
    """Create PG Table from BQ Schema"""
    import asyncio
    import asyncpg
    from google.cloud.sql.connector import Connector

    # Replace the Data Types to work with PostgreSQL supported ones 
    SQL = SQL.replace("object", "TEXT").replace("int64", "INTEGER").replace("float64", "DOUBLE PRECISION")

    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{PROJECT_ID}:{PG_REGION}:{PG_INSTANCE}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{PG_USER}",
            db=f"{PG_DATABASE}",
            password=f"{PG_PASSWORD}"
        )

        await conn.execute(f"DROP SCHEMA IF EXISTS {PG_SCHEMA} CASCADE")        

        await conn.execute(f"CREATE SCHEMA {PG_SCHEMA}")        

        await conn.execute(f"DROP TABLE IF EXISTS {BQ_TABLE} CASCADE")
        
        # Create the table.
        await conn.execute(SQL)

        await conn.close()


# # Create PG Table 
await(create_pg_table(PROJECT_ID, PG_REGION, PG_INSTANCE, PG_PASSWORD, BQ_TABLE, PG_DATABASE, sql,PG_USER))

### F) Import Data to PostgreSQL Table
The below cell will iterate through each export file on our Google Cloud Storage Bucket and load it to the PostgreSQL instance. 
This may take a while, depending on the size of the BigQuery public dataset. You can optionally set the LIMIT parameter to limit how many export files will be loaded in. 

In [28]:
async def import_to_pg(PROJECT_ID,
                          PG_REGION,
                          PG_INSTANCE,
                          PG_USER,
                          PG_PASSWORD,
                          PG_DATABASE,
                          BQ_TABLE, 
                          BUCKET_NAME,
                          field_types): 
    from google.cloud import storage
    import pandas as pd 
    import asyncio
    import asyncpg
    from google.cloud.sql.connector import Connector

    storage_client = storage.Client(project=PROJECT_ID)

    bucket = storage_client.get_bucket(BUCKET_NAME)
    blobs = bucket.list_blobs()

    LIMIT = 3 

    loop = asyncio.get_running_loop()
    async with Connector(loop=loop) as connector:
        # Create connection to Cloud SQL database
        conn: asyncpg.Connection = await connector.connect_async(
            f"{PROJECT_ID}:{PG_REGION}:{PG_INSTANCE}",  # Cloud SQL instance connection name
            "asyncpg",
            user=f"{PG_USER}",
            password=f"{PG_PASSWORD}",
            db=f"{PG_DATABASE}",
        )

        for idx,blob in enumerate(blobs):
            if idx < LIMIT:     # Comment this out if you don't want to use a limit 
                URI = "gs://{}".format(blob.id).split('.csv', 1)[0]+'.csv'
                df = pd.read_csv(URI)

                df = df.dropna()

                df.info()   

                # Copy the dataframe to the table.
                tuples = list(df.itertuples(index=False))

                r = tuples 
                c = list(df) 

                await conn.copy_records_to_table(
                    BQ_TABLE, records=tuples, columns=list(df), schema_name=PG_SCHEMA, timeout=3600
                )
        await conn.close()

    # # Load Data into PG Table 
await(import_to_pg(PROJECT_ID, PG_REGION, PG_INSTANCE, PG_USER, PG_PASSWORD, PG_DATABASE, BQ_TABLE, BUCKET_NAME, field_types))

<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 3 to 107
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   workforce             102 non-null    object 
 1   report_year           102 non-null    int64  
 2   gender_us             102 non-null    object 
 3   race_asian            102 non-null    float64
 4   race_black            102 non-null    float64
 5   race_hispanic_latinx  102 non-null    float64
 6   race_native_american  102 non-null    float64
 7   race_white            102 non-null    float64
dtypes: float64(5), int64(1), object(2)
memory usage: 7.2+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 102 entries, 3 to 107
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   workforce             102 non-null    object 
 1   report_year           102 non-null    int64  
 2   gender_us            

### If all the above steps are executed suucessfully, the Bigquery public dataset should be copied to Cloud SQL for PostgreSQL on your GCP project